In [35]:
import os
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
#from u_interpolate_small import regrid_irregular_quick
from datetime import date
import u_interpolate_small as uint
from ndays import numOfDays
import glob
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cf
import pickle


In [36]:
# Define domain and time period
#end_lat = -7.5
#start_lon = 17.5 #-18
#end_lon = 30
domain ='WA'
if domain=='SA':
    start_lat,end_lat,start_lon,end_lon = -28,-7.5,17.5, 30
    start_year = '2014'
    start_month = '12' #'12'
    end_year = '2015'
    end_month = '02' #'02'
    start_day = '01'
    end_day = '28'

elif domain=='WA':
    start_lat,end_lat,start_lon,end_lon = 8,20,-10,10
    start_year = '2010'
    start_month = '07' #'12'
    end_year = '2010'
    end_month = '09' #'02'
    start_day = '01'
    end_day = '30'

# find ndays
date1 = date(int(start_year), int(start_month), int(start_day))
date2 = date(int(end_year), int(end_month), int(end_day))
t = numOfDays(date1, date2)+1

# define a regular lat/lon grid close to MSG native resolution (0.04 deg)
reg_lat = np.arange(start_lat, end_lat, 0.04)
reg_lon = np.arange(start_lon, end_lon, 0.04) 

In [37]:
# get native MSG grid (core)
lpath = glob.glob('/prj/Africa_cloud/geoloc/lat_lon_2268_2080.npz')[0]  # this is /prj/Africa_cloud/geoloc/*.npz on the Linux system
msg_latlon = np.load(lpath)
mlon = msg_latlon['lon']
mlat = msg_latlon['lat']

In [38]:
# get precip grid and indices using one file
precip_filename = '/prj/global_water/MCS_Feng/global_v2/2d_fields/20100101.0000_20110101.0000/mcstrack_20100901_0630.nc'
precip_data = nc.Dataset(precip_filename,'r')
lat_p_temp = np.array(precip_data.variables['latitude'][:])
lon_p_temp = np.array(precip_data.variables['longitude'][:])
lat_ind_p = np.where((lat_p_temp[:,0]>=start_lat) & (lat_p_temp[:,0]<=end_lat))[0]
lon_ind_p = np.where((lon_p_temp[0,:]>= start_lon) & (lon_p_temp[0,:]<=end_lon))[0]
lat_p = lat_p_temp[lat_ind_p[0]:lat_ind_p[-1]+1,lon_ind_p[0]:lon_ind_p[-1]+1]
lon_p = lon_p_temp[lat_ind_p[0]:lat_ind_p[-1]+1,lon_ind_p[0]:lon_ind_p[-1]+1]

In [39]:
# calculate interpolation weights from MSG (core) and IMERG onto the new regular grid that can be reused to speed up interpolations.
inds, weights, shape = uint.interpolation_weights(mlon, mlat, reg_lon, reg_lat) # save weights for continuous use - MSG interpolation on regular. 

# uncomment line below to include precip interpolation
inds_precip, weights_precip, shape_precip = uint.interpolation_weights(lon_p, lat_p, reg_lon, reg_lat) # save for continuous use - IMERG interpolation weights to same grid. 

In [40]:
# read in precip data 
list_of_file_names_precip=[]

if int(start_year)==int(end_year): 
    dir_name_precip =  '/prj/global_water/MCS_Feng/global_v2/2d_fields/'+start_year+'0101.0000_'+str(int(start_year)+1)+'0101.0000'
    os.chdir(dir_name_precip)
    for m in range(int(start_month),int(end_month)+1,1):
            if m<10:
                month = str(m).zfill(2)
            else:
                month = str(m)   
            all_files=sorted(glob.glob("mcstrack_"+start_year+month+"*.nc"));
            #print(all_files)
            list_of_file_names_precip.append(all_files) # all days in month
             
elif int(start_year)<int(end_year):
    y=1
    for i in range(int(start_year),int(end_year)+1,1):
        dir_name_precip =  '/prj/global_water/MCS_Feng/global_v2/2d_fields/'+str(i)+'0101.0000_'+str(i+1)+'0101.0000/'
        #os.chdir(dir_name_precip)
        if y==1:
            for m in range(int(start_month),12+1,1):
                if m<10:
                    month = str(m).zfill(2)
                else:
                    month = str(m)  
                list_of_file_names_precip.append(sorted(glob.glob(dir_name_precip+'mcstrack_'+str(i)+month+'*.nc'))) # day 1
                y=y+1            
        else:
            for m in range(1,int(end_month)+1,1):
                if m<10:
                    month = str(m).zfill(2)
                else:
                    month = str(m) 
                dir_name_precip =  '/prj/global_water/MCS_Feng/global_v2/2d_fields/'+str(i)+'0101.0000_'+str(i+1)+'0101.0000/'
                list_of_file_names_precip.append(sorted(glob.glob(dir_name_precip+'mcstrack_'+str(i)+month+'*.nc'))) # day 1
                    
    
    #print(dir_name_precip)
list_of_files_precip=[]                
for a in range(0,len(list_of_file_names_precip),1):
    list_of_files_precip = list_of_files_precip+list_of_file_names_precip[a]

In [ ]:
# preallocate
precip = np.zeros((len(lat_p[:,1]),len(lon_p[1,:]),t*24),dtype=float)
regidded_precip = np.zeros((len(reg_lat),len(reg_lon),t*24),dtype=float)*np.nan
time_IMERG = np.zeros(t*24)    
    
for l in range(0,len(list_of_files_precip),1):#22,31,1):#
    precip_filename = list_of_files_precip[l]
    print(precip_filename)
    print(l)
    precip_data = nc.Dataset(precip_filename,'r')
   # precip_temp = np.array(precip_data.variables['precipitation'])[0,:,:]
    precip_temp = np.array(precip_data.variables['precipitation'])[0,lat_ind_p[0]:lat_ind_p[-1]+1,lon_ind_p[0]:lon_ind_p[-1]+1]
    precip[:,:,l] = precip_temp  
    time_IMERG[l]= str(precip_filename[-16:-3])
    regidded_precip[:,:,l] = uint.interpolate_data(precip_temp, inds_precip, weights_precip, shape_precip)  # interpolation using saved weights for MSG TIR


mcstrack_20100701_0030.nc
0
mcstrack_20100701_0130.nc
1
mcstrack_20100701_0230.nc
2
mcstrack_20100701_0330.nc
3
mcstrack_20100701_0430.nc
4
mcstrack_20100701_0530.nc
5
mcstrack_20100701_0630.nc
6
mcstrack_20100701_0730.nc
7
mcstrack_20100701_0830.nc
8
mcstrack_20100701_0930.nc
9
mcstrack_20100701_1030.nc
10
mcstrack_20100701_1130.nc
11
mcstrack_20100701_1230.nc
12
mcstrack_20100701_1330.nc
13
mcstrack_20100701_1430.nc
14
mcstrack_20100701_1530.nc
15
mcstrack_20100701_1630.nc
16
mcstrack_20100701_1730.nc
17
mcstrack_20100701_1830.nc
18
mcstrack_20100701_1930.nc
19
mcstrack_20100701_2030.nc
20
mcstrack_20100701_2130.nc
21
mcstrack_20100701_2230.nc
22
mcstrack_20100701_2330.nc
23
mcstrack_20100702_0030.nc
24
mcstrack_20100702_0130.nc
25
mcstrack_20100702_0230.nc
26
mcstrack_20100702_0330.nc
27
mcstrack_20100702_0430.nc
28
mcstrack_20100702_0530.nc
29
mcstrack_20100702_0630.nc
30
mcstrack_20100702_0730.nc
31
mcstrack_20100702_0830.nc
32
mcstrack_20100702_0930.nc
33
mcstrack_20100702_1030.n

mcstrack_20100712_1330.nc
277
mcstrack_20100712_1430.nc
278
mcstrack_20100712_1530.nc
279
mcstrack_20100712_1630.nc
280
mcstrack_20100712_1730.nc
281
mcstrack_20100712_1830.nc
282
mcstrack_20100712_1930.nc
283
mcstrack_20100712_2030.nc
284
mcstrack_20100712_2130.nc
285
mcstrack_20100712_2230.nc
286
mcstrack_20100712_2330.nc
287
mcstrack_20100713_0030.nc
288
mcstrack_20100713_0130.nc
289
mcstrack_20100713_0230.nc
290
mcstrack_20100713_0330.nc
291
mcstrack_20100713_0430.nc
292
mcstrack_20100713_0530.nc
293
mcstrack_20100713_0630.nc
294
mcstrack_20100713_0730.nc
295
mcstrack_20100713_0830.nc
296
mcstrack_20100713_0930.nc
297
mcstrack_20100713_1030.nc
298
mcstrack_20100713_1130.nc
299
mcstrack_20100713_1230.nc
300
mcstrack_20100713_1330.nc
301
mcstrack_20100713_1430.nc
302
mcstrack_20100713_1530.nc
303
mcstrack_20100713_1630.nc
304
mcstrack_20100713_1730.nc
305
mcstrack_20100713_1830.nc
306
mcstrack_20100713_1930.nc
307
mcstrack_20100713_2030.nc
308
mcstrack_20100713_2130.nc
309
mcstrack_2

mcstrack_20100723_2330.nc
551
mcstrack_20100724_0030.nc
552
mcstrack_20100724_0130.nc
553
mcstrack_20100724_0230.nc
554
mcstrack_20100724_0330.nc
555
mcstrack_20100724_0430.nc
556
mcstrack_20100724_0530.nc
557
mcstrack_20100724_0630.nc
558
mcstrack_20100724_0730.nc
559
mcstrack_20100724_0830.nc
560
mcstrack_20100724_0930.nc
561
mcstrack_20100724_1030.nc
562
mcstrack_20100724_1130.nc
563
mcstrack_20100724_1230.nc
564
mcstrack_20100724_1330.nc
565
mcstrack_20100724_1430.nc
566
mcstrack_20100724_1530.nc
567
mcstrack_20100724_1630.nc
568
mcstrack_20100724_1730.nc
569
mcstrack_20100724_1830.nc
570
mcstrack_20100724_1930.nc
571
mcstrack_20100724_2030.nc
572
mcstrack_20100724_2130.nc
573
mcstrack_20100724_2230.nc
574
mcstrack_20100724_2330.nc
575
mcstrack_20100725_0030.nc
576
mcstrack_20100725_0130.nc
577
mcstrack_20100725_0230.nc
578
mcstrack_20100725_0330.nc
579
mcstrack_20100725_0430.nc
580
mcstrack_20100725_0530.nc
581
mcstrack_20100725_0630.nc
582
mcstrack_20100725_0730.nc
583
mcstrack_2

mcstrack_20100804_0930.nc
825
mcstrack_20100804_1030.nc
826
mcstrack_20100804_1130.nc
827
mcstrack_20100804_1230.nc
828
mcstrack_20100804_1330.nc
829
mcstrack_20100804_1430.nc
830
mcstrack_20100804_1530.nc
831
mcstrack_20100804_1630.nc
832
mcstrack_20100804_1730.nc
833
mcstrack_20100804_1830.nc
834
mcstrack_20100804_1930.nc
835
mcstrack_20100804_2030.nc
836
mcstrack_20100804_2130.nc
837
mcstrack_20100804_2230.nc
838
mcstrack_20100804_2330.nc
839
mcstrack_20100805_0030.nc
840
mcstrack_20100805_0130.nc
841
mcstrack_20100805_0230.nc
842
mcstrack_20100805_0330.nc
843
mcstrack_20100805_0430.nc
844
mcstrack_20100805_0530.nc
845
mcstrack_20100805_0630.nc
846
mcstrack_20100805_0730.nc
847
mcstrack_20100805_0830.nc
848
mcstrack_20100805_0930.nc
849
mcstrack_20100805_1030.nc
850
mcstrack_20100805_1130.nc
851
mcstrack_20100805_1230.nc
852
mcstrack_20100805_1330.nc
853
mcstrack_20100805_1430.nc
854
mcstrack_20100805_1530.nc
855
mcstrack_20100805_1630.nc
856
mcstrack_20100805_1730.nc
857
mcstrack_2

mcstrack_20100815_1530.nc
1095
mcstrack_20100815_1630.nc
1096
mcstrack_20100815_1730.nc
1097
mcstrack_20100815_1830.nc
1098
mcstrack_20100815_1930.nc
1099
mcstrack_20100815_2030.nc
1100
mcstrack_20100815_2130.nc
1101
mcstrack_20100815_2230.nc
1102
mcstrack_20100815_2330.nc
1103
mcstrack_20100816_0030.nc
1104
mcstrack_20100816_0130.nc
1105
mcstrack_20100816_0230.nc
1106
mcstrack_20100816_0330.nc
1107
mcstrack_20100816_0430.nc
1108
mcstrack_20100816_0530.nc
1109
mcstrack_20100816_0630.nc
1110
mcstrack_20100816_0730.nc
1111
mcstrack_20100816_0830.nc
1112
mcstrack_20100816_0930.nc
1113
mcstrack_20100816_1030.nc
1114
mcstrack_20100816_1130.nc
1115
mcstrack_20100816_1230.nc
1116
mcstrack_20100816_1330.nc
1117
mcstrack_20100816_1430.nc
1118
mcstrack_20100816_1530.nc
1119
mcstrack_20100816_1630.nc
1120
mcstrack_20100816_1730.nc
1121
mcstrack_20100816_1830.nc
1122
mcstrack_20100816_1930.nc
1123
mcstrack_20100816_2030.nc
1124
mcstrack_20100816_2130.nc
1125
mcstrack_20100816_2230.nc
1126
mcstrack

In [ ]:
# read in cores data
cores = np.zeros((len(reg_lat),len(reg_lon),t*24),dtype=float)*np.nan #using every fourth value for hourly comparison- 24hr
tir = np.zeros((len(reg_lat),len(reg_lon),t*24),dtype=float)*np.nan #using every fourth value for hourly comparison- 24hr
time_core = np.zeros((t*24)) #using every fourth value for hourly comparison- 24hr

list_of_file_names = []

for i in range(0,len(time_IMERG),1):
    current_file = str(time_IMERG[i])#+'.nc'
    SY= current_file[0:4]
    MM= current_file[4:6]
    all_file_names = '/prj/Africa_cloud/ch9_wavelet/'+SY+'/'+MM+'/'+current_file[0:-2]+'.nc' 
    if os.path.isfile(all_file_names):      
        list_of_file_names.append(all_file_names) # all days in month
    else:
        list_of_file_names.append('file_not_available') # all days in month
            
 
 #print(list_of_file_names)

In [ ]:
cores_temp= np.zeros((len(reg_lat),len(reg_lon)))*np.nan
num_of_missing_core_files = 0

for l in range(0,len(list_of_file_names),1):#2, min(t*96,len(list_of_files)), 4):#range(len(list_of_files)): # 96 files per day
    core_filename = list_of_file_names[l]#'/prj/Africa_cloud/ch9_wavelet/'+str(i)+'/'+month+'/'  list_of_files[l]
    print(core_filename)
    if os.path.isfile(core_filename):      
        ds = xr.open_dataset(core_filename).squeeze() # pick any convective core file from ch9_wavelet. 
        print(l)
        cores[:,:,l] = uint.interpolate_data(ds['cores'].values, inds, weights, shape) # interpolation using saved weights for MSG cores       
        tir[:,:,l] = uint.interpolate_data(ds['tir'].values, inds, weights, shape)  # interpolation using saved weights for MSG TIR      
        time_core[l] = str(core_filename[-15:-3])
    else:
        num_of_missing_core_files = num_of_missing_core_files +1
        cores[:,:,l] = cores_temp
        tir[:,:,l] = cores_temp 
        regidded_precip[:,:,l] = cores_temp  
        time_core[l] = np.nan#str(core_filename[-15:-3])
  

In [ ]:
num_of_missing_core_files


In [ ]:
## a clean way of plotting - use matplotlib functions directly:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
def draw_map(ax, data, lon, lat, title=None,  mask_sig=None, quiver=None, contour=None, cbar_label=None, **kwargs):

    mapp = ax.contourf(lon, lat, data, transform=ccrs.PlateCarree(), **kwargs)  # this is the actual plot
    
    ## mask for significance indicator
    if mask_sig is not None:
         plt.contourf(lon, lat, mask_sig, colors='none', hatches='.',
                     levels=[0.5, 1], linewidth=0.1)
    
    ## quiver list
    if quiver is not None:
        qu = ax.quiver(quiver['x'], quiver['y'], quiver['u'], quiver['v'], scale=quiver['scale'])
    ## additional contour on plot   
    if contour is not None:
        ax.contour(contour['x'], contour['y'], contour['data'], levels=contour['levels'], cmap=contour['cmap'] )
        
    
    ax.coastlines()   ## adds coastlines
    # Gridlines
    xl = ax.gridlines(draw_labels=True);   # adds latlon grid lines
    xl.xlabels_top = False   ## labels off
    xl.ylabels_right = False
    plt.title(title)
    # Countries
    ax.add_feature(cartopy.feature.BORDERS, linestyle='--'); # adds country borders
    cbar = plt.colorbar(mapp)  # adds colorbar
    cbar.set_label(cbar_label)

In [ ]:
day = 24*11

f=plt.figure(figsize=(15,7))  # this opens a plot window
ax = f.add_subplot(111, projection=ccrs.PlateCarree())  # this opens a new plot axis
draw_map(ax, tir[:,:,day], reg_lon, reg_lat, levels=np.arange(-100,50), cbar_label='TIR [Tb oC]', cmap='jet')
ax.contour(reg_lon, reg_lat, cores[:,:,day], levels=[-5,1,50], colors='r')

# ax.set_ylim(-38,28)
# ax.set_xlim(-23,52)

In [ ]:
# plot IMERG rainfall and core data
f=plt.figure(figsize=(15,7))  # this opens a plot window
ax = f.add_subplot(111, projection=ccrs.PlateCarree())  # this opens a new plot axis
draw_map(ax, regidded_precip[:,:,day], reg_lon, reg_lat, levels=np.arange(0,50,5),cbar_label='regridded precip [mm/hr]', cmap='jet')
ax.contour(reg_lon, reg_lat, cores[:,:,day], levels=[-5,1,50], colors='r')

In [ ]:
# plot IMERG rainfall and tir data
f=plt.figure(figsize=(15,7))  # this opens a plot window
ax = f.add_subplot(111, projection=ccrs.PlateCarree())  # this opens a new plot axis
draw_map(ax, regidded_precip[:,:,day], reg_lon, reg_lat, levels=np.arange(0,50,5),cbar_label='regridded precip [mm/hr]', cmap='jet')
ax.contour(reg_lon, reg_lat, tir[:,:,day], levels=np.arange(-100,50,50), cmap='winter')


In [ ]:
# find rainfall pixels collocated with cores
cores_1D = cores.flatten()
TIR_1D = tir.flatten() # cores and precip have same size and indexing - obs in connis notebook
precip_1D = regidded_precip.flatten() # cores and precip have same size and indexing - obs in connis notebook

ind_ok = ~np.isnan(TIR_1D)
TIR_1D = TIR_1D[ind_ok]
cores_1D = cores_1D[ind_ok]
precip_1D = precip_1D[ind_ok]


cores_ind_gt_0 = np.where(cores_1D>0)[0]
precip_collocated_w_cores = precip_1D[cores_ind_gt_0] #dat in connis notebook

print(precip_collocated_w_cores.shape)
 

In [ ]:
print(TIR_1D.shape)

In [ ]:
print(cores_1D.shape)
print(precip_1D.shape)


In [ ]:
#print(np.nanmax(precip_1D))
max_rain_precip_colocated_with_cores = np.nanmax(precip_collocated_w_cores)
max_rain_all_precip = np.nanmax(precip_1D)
print(max_rain_precip_colocated_with_cores)
print(max_rain_all_precip)

In [ ]:
# calculate frequency of rainfall above a threshold 
# dat = 
# threshold = 30mm/hr

def calc_treshReached(dat, thresh):
    return np.sum(np.array(dat)>thresh)/np.sum(np.isfinite(dat))

In [ ]:
# calculate frequency of rainfall above a threshold relative to the same frequency for the full rainfall field (native)

def calc_treshReached_vsObs(dat, obs, thresh):
    return np.sum(np.array(dat)>thresh)/np.sum(np.isfinite(dat)) / np.sum(np.array(obs)>thresh)/np.sum(np.isfinite(obs))

In [ ]:
# calculate fraction of caught rainfall pixels above a threshold
# calc_treshMissed in connis script

def calc_treshcaught(dat, obs, thresh):
    return np.sum(np.array(dat)>thresh)/np.sum(np.array(obs)>thresh)

In [ ]:
# calculate frequency of rainfall above a threshold for regions with wavelet power above a given percentile

def percentile(dat, wav, thresh, perc):
    pval = np.percentile(wav, perc)
    dats = np.array(dat)[np.array(wav)>=pval]
    return np.sum(dats>thresh)/np.sum(np.isfinite(dats))

In [ ]:
# fraction of pixels where threshold exceeded
# dat = precip_collocated_w_cores
# obs = precip_1D
thresh = 20
# treached = calc_treshReached(precip_collocated_w_cores,thresh)
# print(treached)

In [ ]:

# fraction of caught pixels
# tcaught = calc_treshcaught(precip_collocated_w_cores,precip_1D, thresh)
# print(tcaught)

In [ ]:
# fraction of threshold reached in dat vs threshold reached in obs
#tfrac = calc_treshReached_vsObs(precip_collocated_w_cores,precip_1D, thresh)
#print(tfrac)

In [ ]:
# fraction of rainfall pixels reached above intensity threshold to all rainfall pixels (collocated with cores)
f = plt.figure(figsize=(6,4), dpi=200)
ax = f.add_subplot(111)

treached = []
treached1 = []

for intens in range(5,50,5):   
    cc = calc_treshReached(precip_collocated_w_cores, intens)
    cc1 = calc_treshReached(precip_1D, intens)
    treached.append(cc)
    treached1.append(cc1)

ax.plot(range(5,50,5), treached,color='green',label='Precip collocated w/ cores')
ax.plot(range(5,50,5), treached1,color='red',label='IMERG precip')

ax.set_title("Intensity probability (%) (x[treshhold]/x)")
plt.xlabel('intensity of rainfall [mm/hr]')
plt.legend()

             

In [ ]:
# fraction of rainfall pixels reached above intensity threshold to all rainfall pixels (collocated with cores)precip
precip_ind_gt_1 = np.where(precip_collocated_w_cores>1)[0] #find pixels with > 1mm precip
precip_collocated_w_cores_gt_1 = precip_collocated_w_cores[precip_ind_gt_1] #dat in connis notebook
precip_ind_gt_1 = np.where(precip_1D>1)[0] #find pixels with > 1mm precip
precip_1D_gt_1 = precip_1D[precip_ind_gt_1] 


# Open a file and use dump() 
with open('/users/global/jawahm/Documents/Africa_cores/stats/'+start_year+domain+'.pkl', 'wb') as file:       
    # A new file will be created 
    pickle.dump([precip_collocated_w_cores_gt_1,precip_1D_gt_1], file) 
    

f = plt.figure(figsize=(6,4), dpi=200)
ax = f.add_subplot(111)

treached = []
treached1 = []

for intens in range(5,50,5):   
    cc = calc_treshReached(precip_collocated_w_cores_gt_1, intens)
    cc1 = calc_treshReached(precip_1D_gt_1, intens)
    treached.append(cc)
    treached1.append(cc1)

ax.plot(range(5,50,5), treached,color='green',label='Precip collocated w/ cores')
ax.plot(range(5,50,5), treached1,color='red',label='IMERG precip')

ax.set_title("Precip>1mm only- Intensity probability (%) (x[treshhold]/x)")
plt.xlabel('intensity of rainfall [mm/hr]')
plt.legend()

# Open a file and use dump() 
#with open('/users/global/jawahm/Documents/Africa_cores/stats/'+start_year+'_intensity_'+domain+'.pkl', 'wb') as file: 
      
    # A new file will be created 
   # pickle.dump([treached,treached1], file) 
    

In [ ]:
# plot frequency of caught pixels with rainfall above a threshold for regions with wavelet power above a given 
# percentile
f = plt.figure(figsize=(6,4), dpi=200)
ax = f.add_subplot(111)

treached = []
for intens in range(5,50,5):   
    cc = percentile(precip_collocated_w_cores, cores_1D[cores_ind_gt_0], intens, 90)
    treached.append(cc)
    #plotdic.append((treached,dat[1]))
#for pp in plotdic:    
#ax.plot(range(5,50,5), pp[0], label=pp[1])

ax.plot(range(5,50,5), treached,color='green')
ax.set_title("Intensity probability for 90th centile power (%) (x[treshhold]/x)")
plt.xlabel('intensity of rainfall [mm/hr]')
#ax.ylabel('fraction of rainfall pixels caught above intensity threshold that have >= 75th percentile wavelet power')
#plt.legend()
#f.savefig("/home/ck/DIR/cornkle/data/nflics/core_tests/"+'rainP_75p_wpower.jpg') 

# Open a file and use dump() 
with open('/users/global/jawahm/Documents/Africa_cores/stats/'+start_year+'_intensity_90_'+domain+'.pkl', 'wb') as file: 
      
    # A new file will be created 
    pickle.dump(treached, file) 
 

In [ ]:
# fraction of rainfall pixels caught above intensity threshold to all caught rainfall pixels
f = plt.figure(figsize=(6,4), dpi=200)
ax = f.add_subplot(111)

tcaught = []
retained_core_sample_size=[]
retained_IMERG_sample_size=[]

for intens in range(5,50,5):   
    cc = calc_treshcaught(precip_collocated_w_cores_gt_1, precip_1D_gt_1, intens)
#    cc = calc_treshcaught(precip_collocated_w_cores, precip_1D, intens)
    tcaught.append(cc)
    dummy1 = np.where(np.array(precip_collocated_w_cores_gt_1)>intens)
    dummy2 = np.where(np.array(precip_1D_gt_1)>intens)
    retained_core_sample_size.append(len(dummy1[0]))
    retained_IMERG_sample_size.append(len(dummy2[0]))

ax.plot(range(5,50,5), tcaught,color='red')
ax.set_title("Retained pixels per intensity (%)")
plt.xlabel('intensity of rainfall [mm/hr]')
f.savefig('/users/global/jawahm/Documents/Africa_cores/figures/'+start_year+'_retained_pixels_'+domain+'.jpg')  

# Open a file and use dump() 
with open('/users/global/jawahm/Documents/Africa_cores/stats/'+start_year+'_retained_pixels_'+domain+'.pkl', 'wb') as file:      
    # A new file will be created 
    pickle.dump(tcaught, file) 
 
with open('/users/global/jawahm/Documents/Africa_cores/stats/'+start_year+'_retained_pixels_sample_size_'+domain+'.pkl', 'wb') as file:      
    # A new file will be created 
    pickle.dump([retained_core_sample_size,retained_IMERG_sample_size], file) 

In [ ]:
with open('/users/global/jawahm/Documents/Africa_cores/stats/'+start_year+'_max_rain_'+domain+'.pkl', 'wb') as file: 
    pickle.dump([max_rain_precip_colocated_with_cores,max_rain_all_precip],file)
